# CS4248 Project
(DON'T CLICK RUN ALL! Lemmatization takes tooo long and pls don't run it again)

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# 
# %cd "/content/drive/My Drive/CS4248 Project"
# !cd "/content/drive/My Drive/CS4248 Project"

In [2]:
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
csv.field_size_limit(999999)
train = pd.read_csv('raw_data/fulltrain.csv', header = None, names=['class','text'])
X_train = train['text']
y_train = train['class']

In [4]:
train.head()

,class,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


In [5]:
test = pd.read_csv("raw_data/balancedtest.csv", header = None, names=['class','text'])
X_test = test['text']
y_test = test['class']

In [6]:
test.head()

,class,text
0,1,When so many actors seem content to churn out ...
1,1,In what football insiders are calling an unex...
2,1,In a freak accident following Game 3 of the N....
3,1,North Koreas official news agency announced to...
4,1,The former Alaska Governor Sarah Palin would b...


## Baseline Models

In [7]:
def tfidf(X_train, X_test):

  tfidf_baseline = TfidfVectorizer(ngram_range=(1,1), max_features = 10000)

  X_train_tfidf = tfidf_baseline.fit_transform(X_train)

  X_test_tfidf = tfidf_baseline.transform(X_test)

  return X_train_tfidf, X_test_tfidf

X_train_tfidf, X_test_tfidf = tfidf(X_train, X_test)

In [8]:
# Logistic Regression - baseline

def LR(X_train_tfidf, y_train, X_test_tfidf):

  LR_classifier = LogisticRegression(random_state = 42, max_iter=1000).fit(X_train_tfidf, y_train)

  y_pred_lr = LR_classifier.predict(X_test_tfidf)

  print(classification_report(y_test, y_pred_lr))

  return y_pred_lr

y_pred_lr = LR(X_train_tfidf, y_train, X_test_tfidf)

              precision    recall  f1-score   support

           1       0.85      0.79      0.82       750
           2       0.81      0.37      0.51       750
           3       0.56      0.83      0.67       750
           4       0.81      0.92      0.86       750

    accuracy                           0.73      3000
   macro avg       0.76      0.73      0.71      3000
weighted avg       0.76      0.73      0.71      3000


In [9]:
# NB - baseline

def NB(X_train_tfidf, y_train, X_test_tfidf):

  nb_classifier = ComplementNB().fit(X_train_tfidf, y_train)

  y_pred_nb = nb_classifier.predict(X_test_tfidf)

  print(classification_report(y_test, y_pred_nb))

  return y_pred_nb

y_pred_nb = NB(X_train_tfidf, y_train, X_test_tfidf)

              precision    recall  f1-score   support

           1       0.62      0.61      0.61       750
           2       0.62      0.46      0.53       750
           3       0.62      0.93      0.74       750
           4       0.88      0.67      0.76       750

    accuracy                           0.67      3000
   macro avg       0.68      0.67      0.66      3000
weighted avg       0.68      0.67      0.66      3000


## Preprocessing

### Lemmatization with POS Tagging
(DON'T RUN AGAIN! It takes tooooo long)

In [10]:

# WORDNET LEMMATIZER (with appropriate pos tags)
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

# Define function to lemmatize each word with its POS tag

# POS_TAGGER_FUNCTION : TYPE 1
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_with_pos(tokens):
  # tokenize the sentence and find the POS tag for each token
  pos_tagged = nltk.pos_tag(tokens)

  # print(pos_tagged)

  wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
  # print(wordnet_tagged)

  lemmatized_sentence = []
  for word, tag in wordnet_tagged:
      if tag is None:
          # if there is no available tag, append the token as is
          lemmatized_sentence.append(word)
      else:
          # else use the tag to lemmatize the token
          lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
  # lemmatized_sentence = " ".join(lemmatized_sentence)

  # print(lemmatized_sentence)
  return lemmatized_sentence

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/stella/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [11]:
# Start preprocessing

from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from tqdm import tqdm
!pip install pyspellchecker
from spellchecker import SpellChecker

punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

spell = SpellChecker()

def preprocessing(data):
  data_clean = []
  for sentence in tqdm(data):

    # Tokenization
    tokens = word_tokenize(sentence)

    # Remove punctuation and number
    tokens = [w for w in tokens if (not w in punc) and (not w.isdigit())]

    # Spell check
    # Taking too long time for each sentence, not practical to be used
    # tokens = [spell.correction(w) for w in tokens]
    # tokens = [w for w in tokens if w is not None and len(w) > 0]

    # Lemmatization based on tagging
    tokens = lemmatize_with_pos(tokens)

    data_clean.append((' ').join(tokens))
  return data_clean


X_train_clean = preprocessing(X_train)
X_test_clean = preprocessing(X_test)

X_train_clean[:2]

[nltk_data] Downloading package punkt to /Users/stella/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/stella/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


100%|██████████| 3000/3000 [00:36<00:00, 81.40it/s]


['A little less than a decade ago hockey fan be bless with a slate of game every night but on Thursday source confirm that for the ninth consecutive year NHL player have be lock out with very slim hope of an agreement in sight It seem like just yesterday Martin St. Louis and his Lightning teammate be raise the Stanley Cup high school hockey coach and onetime ESPN analyst Barry Melrose say Obviously Im still hop the two side can come together and reach an agreement but Im start to think nobody really miss hockey anymore Nope Nobody but old Barry Id still love to catch an Atlanta Thrashers game Observers have note that when arena door do reopen the NHL will face the perhaps great challenge of convince fan to return to hockey instead of watch more popular sport like football basketball baseball and SlamBall',
 "The writer of the HBO series The Sopranos take another dare storytelling step by kill off million fan during the seventh season 's premiere episode Sunday night 'This be definitely

In [12]:
X_train_clean_tfidf, X_test_clean_tfidf = tfidf(X_train_clean, X_test_clean)

_ = LR(X_train_clean_tfidf, y_train, X_test_clean_tfidf)

              precision    recall  f1-score   support

           1       0.85      0.81      0.83       750
           2       0.84      0.42      0.56       750
           3       0.59      0.83      0.69       750
           4       0.81      0.92      0.86       750

    accuracy                           0.74      3000
   macro avg       0.77      0.75      0.74      3000
weighted avg       0.77      0.74      0.74      3000


In [13]:
_ = NB(X_train_clean_tfidf, y_train, X_test_clean_tfidf)

              precision    recall  f1-score   support

           1       0.60      0.61      0.61       750
           2       0.65      0.51      0.57       750
           3       0.63      0.92      0.75       750
           4       0.87      0.64      0.74       750

    accuracy                           0.67      3000
   macro avg       0.69      0.67      0.67      3000
weighted avg       0.69      0.67      0.67      3000


## Feature Engineering

In [14]:
# Text Subjectivity (w/o preprocessing)
from textblob import TextBlob

def subjectivity(data, tf_idf):
    # new_data = data.copy()
    new_data = data
    new_data['psycho-linguistic'] = new_data["text"].apply(lambda x: TextBlob(x).sentiment)
    new_data['subjectivity'] = new_data['psycho-linguistic'].apply(lambda x: x[1])
    tfidf_df = pd.DataFrame(tf_idf.toarray())
    subject_df = pd.merge(tfidf_df, new_data['subjectivity'],left_index=True, right_index=True)
    subject_df.columns = subject_df.columns.astype(str)
    return subject_df

train_subject_df = subjectivity(train, X_train_tfidf)
test_subject_df = subjectivity(test, X_test_tfidf)

In [15]:
_ = LR(train_subject_df, y_train, test_subject_df)

              precision    recall  f1-score   support

           1       0.85      0.81      0.83       750
           2       0.81      0.37      0.51       750
           3       0.57      0.82      0.67       750
           4       0.81      0.92      0.86       750

    accuracy                           0.73      3000
   macro avg       0.76      0.73      0.72      3000
weighted avg       0.76      0.73      0.72      3000


In [16]:
_ = NB(train_subject_df, y_train, test_subject_df)

              precision    recall  f1-score   support

           1       0.62      0.62      0.62       750
           2       0.62      0.46      0.53       750
           3       0.62      0.93      0.74       750
           4       0.88      0.67      0.76       750

    accuracy                           0.67      3000
   macro avg       0.68      0.67      0.66      3000
weighted avg       0.68      0.67      0.66      3000


In [17]:
# Text Subjectivity (with preprocessing)
train_clean_subject_df = subjectivity(train, X_train_clean_tfidf)
test_clean_subject_df = subjectivity(test, X_test_clean_tfidf)

In [18]:
_ = LR(train_clean_subject_df, y_train, test_clean_subject_df)

              precision    recall  f1-score   support

           1       0.86      0.83      0.84       750
           2       0.83      0.40      0.54       750
           3       0.58      0.83      0.69       750
           4       0.81      0.92      0.86       750

    accuracy                           0.74      3000
   macro avg       0.77      0.74      0.73      3000
weighted avg       0.77      0.74      0.73      3000


In [19]:
_ = NB(train_clean_subject_df, y_train, test_clean_subject_df)

              precision    recall  f1-score   support

           1       0.60      0.61      0.61       750
           2       0.64      0.51      0.57       750
           3       0.63      0.92      0.75       750
           4       0.87      0.65      0.74       750

    accuracy                           0.67      3000
   macro avg       0.69      0.67      0.67      3000
weighted avg       0.69      0.67      0.67      3000


In [27]:
# Hedging Word Count (w/o preprocessing)
#source: http://jal.iaut.ac.ir/article_524171_19ce974d4c9441a312df2d6eaac20a15.pdf
#https://hal.inria.fr/hal-03342756/file/Besancon__Definitely_Maybe__preprint%20%281%29.pdf
import re
modal = "Can Could May Might Should Will Would"
re_modal = "(?:^|\W)(" + "|".join(modal.lower().split(" ")) + ")(?:$|\W)"

lexical = "Appear Argue Assert Assume Attempt Believe Claim Conclude Consider Doubt Estimate Evaluate Expect Feel Hypothesize Imply Indicate Interpret Maintain Note Offer Observe Perceive Presume Propose Prove Report Seem Seen Speculate Suggest Suppose Suspect Tend Think"
lexical_s = [i + "s" for i in lexical.lower().split(" ")]
re_lexical = "(?:^|\W)(" + "|".join(lexical.lower().split(" ") + lexical_s) + "|attempt to|can be seen" + ")(?:$|\W)"

adverbs = "About Allegedly Almost Apparently Approximately Around Arguably Barely Commonly Conceivably Considerably Doubtless Fairly Frequently Generally Given that Greatly Highly Hypothetically Largely Likely Mainly Markedly Maybe Modestly Mostly Nearly Normally Occasionally Often Partially Partly Perhaps Possibly Potentially Practically Presumably Primarily Probably Provided Quite Rarely Reasonably Relatively Reportedly Roughly Seemingly Seldom Significantly Slightly Sometimes Somewhat Strongly Substantially Supposedly Tentatively Theoretically Typically Unlikely Usually Vastly Virtually Widely"
re_adverbs = "(?:^|\W)(" + "|".join(adverbs.lower().split(" ")) + "|provided that" + ")(?:$|\W)"

adjectives = "Apparent Approximate Common Conceivable Considerable Consistent with Frequent General Hypothetical Improbable Indicative Large Likely Little Main Major Modest Noticeable Plausible Possible Potential Primary Probable Rare Relative Remarkable Rough Significant Slight Small Substantial Theoretical Typical Uncommon Unlikely Usual Well-Known"
re_adjectives = "(?:^|\W)(" + "|".join(adjectives.lower().split(" ")) + ")(?:$|\W)"

nouns = "Alternative Approximation Assertion Assessment Assumption Belief Chance Claim Conclusion Doubt Estimate Estimation Evaluation Expectation Hope Idea Implication Indication Interpretation Likelihood Opinion Possibility Premise Probability Proposal Suggestion Tendency View"
re_nouns = "(?:^|\W)(" + "|".join(nouns.lower().split(" ")) + ")(?:$|\W)"

In [28]:
def hedges(x):
    if (len(re.findall(re_modal,x.lower())) +
            len(re.findall(re_lexical,x.lower())) +
            len(re.findall(re_adverbs,x.lower())) +
            len(re.findall(re_adjectives,x.lower())) +
            len(re.findall(re_nouns,x.lower()))) == 0:
        return 0
    else:
        return 1

def hedging_word_count(data, tf_idf):
    new_data = data.copy()
    new_data["hedges"] = new_data.loc[:, "text"].apply(hedges)
    tfidf_df = pd.DataFrame(tf_idf.toarray())
    hedged_df = pd.merge(tfidf_df, new_data['hedges'],left_index=True, right_index=True)
    hedged_df.columns = hedged_df.columns.astype(str)
    return hedged_df

train_hedge_df = hedging_word_count(train, X_train_tfidf)
test_hedge_df = hedging_word_count(test, X_test_tfidf)

In [29]:
_ = LR(train_hedge_df, y_train, test_hedge_df)

              precision    recall  f1-score   support

           1       0.84      0.82      0.83       750
           2       0.80      0.36      0.50       750
           3       0.57      0.83      0.68       750
           4       0.81      0.92      0.86       750

    accuracy                           0.73      3000
   macro avg       0.75      0.73      0.72      3000
weighted avg       0.75      0.73      0.72      3000


In [30]:
_ = NB(train_hedge_df, y_train, test_hedge_df)

              precision    recall  f1-score   support

           1       0.62      0.62      0.62       750
           2       0.62      0.46      0.53       750
           3       0.62      0.93      0.74       750
           4       0.88      0.67      0.76       750

    accuracy                           0.67      3000
   macro avg       0.69      0.67      0.66      3000
weighted avg       0.69      0.67      0.66      3000


In [31]:
# Hedging Word Count (with preprocessing)
train_clean_hedge_df = hedging_word_count(train, X_train_clean_tfidf)
test_clean_hedge_df = hedging_word_count(test, X_test_clean_tfidf)

In [32]:
_ = LR(train_clean_hedge_df, y_train, test_clean_hedge_df)

              precision    recall  f1-score   support

           1       0.85      0.83      0.84       750
           2       0.82      0.41      0.54       750
           3       0.59      0.83      0.69       750
           4       0.80      0.91      0.85       750

    accuracy                           0.74      3000
   macro avg       0.77      0.74      0.73      3000
weighted avg       0.77      0.74      0.73      3000


In [33]:
_ = NB(train_clean_hedge_df, y_train, test_clean_hedge_df)

              precision    recall  f1-score   support

           1       0.60      0.61      0.61       750
           2       0.64      0.51      0.57       750
           3       0.63      0.91      0.75       750
           4       0.87      0.64      0.74       750

    accuracy                           0.67      3000
   macro avg       0.69      0.67      0.67      3000
weighted avg       0.69      0.67      0.67      3000
